# Importação de bibliotecas

In [35]:
import pandas as pd
from sqlalchemy import create_engine

# Instância da conexão com o banco de dados

In [36]:
usuario = "root"
senha = "root"
url = "localhost"
porta = "3306"
banco = "ENEM_DM"

# Criar uma conexão com o banco de dados MySQL
engine_dm = create_engine(f"mysql+pymysql://{usuario}:{senha}@{url}:{porta}/{banco}")

# 1 - Consulta SQL para obter a média da nota de redação agrupado por cor/raça e sexo

Apenas são consideradas as pessoas que não tiveram a redação anulada.

In [37]:
query = """
  WITH CTE AS (
      SELECT
        COR_RACA,
        SEXO,
        AVG(
          NU_NOTA_REDACAO
        ) AS MEDIA_REDACAO
      FROM
        FAT_NOTAS
        JOIN DIM_PARTICIPANTES USING (SK_PARTICIPANTES)
        JOIN DIM_SEXO USING (SK_SEXO)
        JOIN DIM_COR_RACA USING (SK_COR_RACA)
        JOIN DIM_STATUS_REDACAO USING (SK_STATUS_REDACAO)
    WHERE
      TP_STATUS_REDACAO = 1
    GROUP BY
      COR_RACA,
      SEXO
  )
  SELECT
    *,
    RANK() OVER (
      ORDER BY
        MEDIA_REDACAO DESC
    ) AS RNK
  FROM
    CTE
  ORDER BY
    RNK
"""

df = pd.read_sql_query(query, con=engine_dm)

display(df)

,COR_RACA,SEXO,MEDIA_REDACAO,RNK
0,Branca,Feminino,709.38498,1
1,Branca,Masculino,672.62476,2
2,Não declarado,Feminino,655.93132,3
3,Parda,Feminino,643.89233,4
4,Amarela,Feminino,636.62697,5
5,Amarela,Masculino,629.34235,6
6,Preta,Feminino,628.16692,7
7,Parda,Masculino,619.97137,8
8,Não declarado,Masculino,617.61613,9
9,Preta,Masculino,603.67247,10


# 2 - Consulta SQL para obter a média das provas agrupado por cor/raça e sexo

Apenas são consideradas as pessoas que tiveram presenças nas provas.

Provas consideradas:
- Linguagens e Códigos
- Matemática
- Ciências da Natureza
- Ciências Humanas

In [38]:
query = """
  WITH CTE AS (
      SELECT
        COR_RACA,
        SEXO,
        AVG(MEDIA_PROVA) AS MEDIA_PROVA
      FROM
        (
          SELECT
            COR_RACA,
            SEXO,
            (NU_NOTA_CN + NU_NOTA_CH + NU_NOTA_LC + NU_NOTA_MT) / 4 AS MEDIA_PROVA
          FROM
            FAT_NOTAS
            JOIN DIM_PARTICIPANTES USING (SK_PARTICIPANTES)
            JOIN DIM_SEXO USING (SK_SEXO)
            JOIN DIM_COR_RACA USING (SK_COR_RACA)
            JOIN DIM_PRESENCA_CN USING (SK_PRESENCA_CN)
            JOIN DIM_PRESENCA_CH USING (SK_PRESENCA_CH)
            JOIN DIM_PRESENCA_LC USING (SK_PRESENCA_LC)
            JOIN DIM_PRESENCA_MT USING (SK_PRESENCA_MT)
          WHERE
            1 = 1
            AND TP_PRESENCA_CN = 1
            AND TP_PRESENCA_CH = 1
            AND TP_PRESENCA_LC = 1
            AND TP_PRESENCA_MT = 1
        ) SUB
      GROUP BY
        COR_RACA,
        SEXO
  )
  SELECT
    *,
    RANK() OVER (
      ORDER BY
        MEDIA_PROVA DESC
    ) AS RNK
  FROM
    CTE
  ORDER BY
    RNK
"""

df = pd.read_sql_query(query, con=engine_dm)

display(df)

,COR_RACA,SEXO,MEDIA_PROVA,RNK
0,Branca,Masculino,555.825692,1
1,Branca,Feminino,535.932547,2
2,Amarela,Masculino,528.864623,3
3,Não declarado,Masculino,525.041817,4
4,Parda,Masculino,513.880787,5
5,Não declarado,Feminino,511.741225,6
6,Preta,Masculino,504.113358,7
7,Amarela,Feminino,498.596824,8
8,Parda,Feminino,493.655424,9
9,Preta,Feminino,489.924821,10


# 3 - Consulta SQL para obter a média da prova de matemática agrupado por cor/raça, sexo e estado civil

Apenas são consideradas as pessoas que tiveram presença na prova.

In [39]:
query = """
  SELECT
    COR_RACA,
    SEXO,
    ESTADO_CIVIL,
    COALESCE(AVG(NU_NOTA_MT), 0) AS MEDIA_MT,
    RANK() OVER (
      ORDER BY
        COALESCE(AVG(NU_NOTA_MT), 0) DESC
    ) AS RNK
  FROM
    FAT_NOTAS
    JOIN DIM_PARTICIPANTES USING (SK_PARTICIPANTES)
    JOIN DIM_SEXO USING (SK_SEXO)
    JOIN DIM_COR_RACA USING (SK_COR_RACA)
    JOIN DIM_ESTADO_CIVIL USING (SK_ESTADO_CIVIL)
    JOIN DIM_PRESENCA_MT USING (SK_PRESENCA_MT)
  WHERE
    TP_PRESENCA_MT = 1
  GROUP BY
    COR_RACA,
    SEXO,
    ESTADO_CIVIL
  ORDER BY
    RNK
  LIMIT
    10
"""

df = pd.read_sql_query(query, con=engine_dm)

display(df)

,COR_RACA,SEXO,ESTADO_CIVIL,MEDIA_MT,RNK
0,Não declarado,Masculino,Viúvo(a),654.50000,1
1,Branca,Masculino,Solteiro(a),598.10479,2
2,Branca,Masculino,Não informado,576.26460,3
3,Não declarado,Masculino,Casado(a)/Mora com companheiro(a),570.72821,4
4,Branca,Masculino,Divorciado(a)/Desquitado(a)/Separado(a),569.94752,5
5,Amarela,Masculino,Solteiro(a),565.92747,6
6,Branca,Masculino,Casado(a)/Mora com companheiro(a),565.29842,7
7,Branca,Feminino,Solteiro(a),553.81399,8
8,Não declarado,Masculino,Solteiro(a),550.46665,9
9,Amarela,Masculino,Divorciado(a)/Desquitado(a)/Separado(a),545.71538,10


# 4 - Consulta SQL para obter as piores notas de prova de Linguages e Código agrupado por nacionalidade, faixa etária e ensino

Apenas são consideradas as pessoas que tiveram presença na prova.

In [40]:
query = """
  SELECT
    NACIONALIDADE,
    FAIXA_ETARIA,
    ENSINO,
    FIRST_VALUE (COALESCE(AVG(NU_NOTA_LC), 0)) OVER (
      PARTITION BY
        FAIXA_ETARIA
      ORDER BY
        AVG(NU_NOTA_LC) ASC
    ) AS PIOR_NOTA
  FROM
    FAT_NOTAS
    JOIN DIM_PARTICIPANTES USING (SK_PARTICIPANTES)
    JOIN DIM_FAIXA_ETARIA USING (SK_FAIXA_ETARIA)
    JOIN DIM_NACIONALIDADE USING (SK_NACIONALIDADE)
    JOIN DIM_ENSINO USING (SK_ENSINO)
    JOIN DIM_PRESENCA_LC USING (SK_PRESENCA_LC)
  WHERE
    TP_PRESENCA_LC = 1
  GROUP BY
    NACIONALIDADE,
    FAIXA_ETARIA,
    ENSINO
  ORDER BY
    PIOR_NOTA
  LIMIT
    10
"""

df = pd.read_sql_query(query, con=engine_dm)

display(df)

,NACIONALIDADE,FAIXA_ETARIA,ENSINO,PIOR_NOTA
0,Não informado,23 anos,Ensino Regular,301.8
1,Brasileiro(a),23 anos,Educação Especial - Modalidade Substitutiva,301.8
2,Brasileiro(a) Naturalizado(a),23 anos,Ensino Regular,301.8
3,Brasileiro(a),23 anos,Ensino Regular,301.8
4,Brasileiro(a),18 anos,Ensino Regular,303.2
5,Estrangeiro(a),18 anos,Ensino Regular,303.2
6,Brasileiro(a) Naturalizado(a),18 anos,Ensino Regular,303.2
7,Brasileiro(a) Naturalizado(a),18 anos,Educação Especial - Modalidade Substitutiva,303.2
8,Brasileiro(a),18 anos,Educação Especial - Modalidade Substitutiva,303.2
9,Estrangeiro(a),18 anos,Educação Especial - Modalidade Substitutiva,303.2


# 5 - Consulta SQL para obter a média de pessoas que moram com os participantes e da redação, filtrando 10.000 participantes com as melhores notas de redação e agrupado por sexo e estado civil

Apenas são consideradas as pessoas que não tiveram a redação anulada.

In [45]:
query = """
  WITH AUX AS (
    SELECT
      SEXO,
      ESTADO_CIVIL,
      RESP_QUESTAO_5,
      NU_NOTA_REDACAO,
      ROW_NUMBER() OVER (
        PARTITION BY
          SEXO,
          ESTADO_CIVIL
        ORDER BY
          NU_NOTA_REDACAO DESC
      ) AS RN
    FROM
      FAT_NOTAS
      JOIN FAT_QUESTIONARIO USING (SK_PARTICIPANTES)
      JOIN DIM_RESP_QUESTAO_5 USING (SK_RESP_QUESTAO_5)
      JOIN DIM_PARTICIPANTES USING (SK_PARTICIPANTES)
      JOIN DIM_SEXO USING (SK_SEXO)
      JOIN DIM_COR_RACA USING (SK_COR_RACA)
      JOIN DIM_ESTADO_CIVIL USING (SK_ESTADO_CIVIL)
      JOIN DIM_STATUS_REDACAO USING (SK_STATUS_REDACAO)
    WHERE
      TP_STATUS_REDACAO = 1
      AND COR_RACA = 'Branca'
  )
  SELECT
    SEXO,
    ESTADO_CIVIL,
    ROUND(AVG(NU_NOTA_REDACAO), 1) AS MEDIA_REDACAO,
    ROUND(AVG(RESP_QUESTAO_5), 2) as MEDIA_PESSOAS_RESIDENCIA
  FROM
    AUX
  WHERE
    RN <= 10000
  GROUP BY
    SEXO,
    ESTADO_CIVIL
  ORDER BY
	  MEDIA_PESSOAS_RESIDENCIA DESC
"""

df = pd.read_sql_query(query, con=engine_dm)

display(df)

,SEXO,ESTADO_CIVIL,MEDIA_REDACAO,MEDIA_PESSOAS_RESIDENCIA
0,Masculino,Solteiro(a),935.1,3.63
1,Feminino,Não informado,664.6,3.60
2,Feminino,Solteiro(a),959.9,3.60
3,Masculino,Não informado,640.3,3.59
4,Masculino,Casado(a)/Mora com companheiro(a),585.0,3.30
5,Feminino,Casado(a)/Mora com companheiro(a),587.5,3.27
6,Masculino,Divorciado(a)/Desquitado(a)/Separado(a),609.7,3.16
7,Feminino,Divorciado(a)/Desquitado(a)/Separado(a),596.4,3.12
8,Feminino,Viúvo(a),518.6,2.71
9,Masculino,Viúvo(a),512.4,2.62
